# Treniranje RL-a za balansiranje Cartpolea

Ova bilježnica dio je [kurikuluma AI za početnike](http://aka.ms/ai-beginners). Inspirirana je [službenim PyTorch vodičem](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) i [ovom PyTorch implementacijom za Cartpole](https://github.com/yc930401/Actor-Critic-pytorch).

U ovom primjeru koristit ćemo RL za treniranje modela koji balansira štap na kolicima koja se mogu kretati lijevo i desno na horizontalnoj skali. Koristit ćemo okruženje [OpenAI Gym](https://www.gymlibrary.ml/) za simulaciju štapa.

> **Napomena**: Kod ove lekcije možete pokrenuti lokalno (npr. iz Visual Studio Code-a), u kojem će se slučaju simulacija otvoriti u novom prozoru. Ako kod pokrećete online, možda ćete trebati napraviti neke prilagodbe koda, kako je opisano [ovdje](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Započet ćemo tako da provjerimo je li Gym instaliran:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Sada ćemo stvoriti CartPole okruženje i vidjeti kako njime upravljati. Okruženje ima sljedeće značajke:

* **Prostor akcija** je skup mogućih akcija koje možemo izvršiti u svakom koraku simulacije
* **Prostor opažanja** je prostor opažanja koje možemo napraviti


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Pogledajmo kako simulacija funkcionira. Sljedeća petlja pokreće simulaciju dok `env.step` ne vrati zastavicu za završetak `done`. Akcije ćemo nasumično birati pomoću `env.action_space.sample()`, što znači da će eksperiment vjerojatno vrlo brzo propasti (okruženje CartPole završava kada brzina CartPole-a, njegova pozicija ili kut budu izvan određenih granica).

> Simulacija će se otvoriti u novom prozoru. Kod možete pokrenuti nekoliko puta i vidjeti kako se ponaša.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Možete primijetiti da opažanja sadrže 4 broja. Oni su:
- Položaj kolica
- Brzina kolica
- Kut stupa
- Brzina rotacije stupa

`rew` je nagrada koju primamo pri svakom koraku. Možete vidjeti da u CartPole okruženju dobivate 1 bod za svaki korak simulacije, a cilj je maksimizirati ukupnu nagradu, tj. vrijeme tijekom kojeg CartPole može održavati ravnotežu bez pada.

Tijekom učenja pojačanjem, naš cilj je trenirati **politiku** $\pi$, koja će za svako stanje $s$ odrediti koju akciju $a$ trebamo poduzeti, dakle u suštini $a = \pi(s)$.

Ako želite probabilističko rješenje, možete razmišljati o politici kao o vraćanju skupa vjerojatnosti za svaku akciju, tj. $\pi(a|s)$ bi značilo vjerojatnost da trebamo poduzeti akciju $a$ u stanju $s$.

## Metoda gradijenta politike

U najjednostavnijem algoritmu za učenje pojačanjem, nazvanom **Gradijent politike**, trenirat ćemo neuronsku mrežu da predvidi sljedeću akciju.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Trenirat ćemo mrežu izvođenjem mnogih eksperimenata i ažuriranjem naše mreže nakon svakog izvođenja. Definirajmo funkciju koja će izvršiti eksperiment i vratiti rezultate (tzv. **trag**) - sva stanja, akcije (i njihove preporučene vjerojatnosti) i nagrade:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Možete pokrenuti jednu epizodu s neuvježbanom mrežom i primijetiti da je ukupna nagrada (poznata i kao duljina epizode) vrlo niska:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Jedan od izazovnih aspekata algoritma gradijenta politike je korištenje **diskontiranih nagrada**. Ideja je da izračunamo vektor ukupnih nagrada u svakom koraku igre, a tijekom tog procesa diskontiramo rane nagrade koristeći neki koeficijent $gamma$. Također normaliziramo dobiveni vektor, jer ćemo ga koristiti kao težinu za utjecaj na naše treniranje:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Sada krenimo s pravim treningom! Izvest ćemo 300 epizoda, a u svakoj epizodi napravit ćemo sljedeće:

1. Provesti eksperiment i prikupiti trag.
2. Izračunati razliku (`gradients`) između poduzetih akcija i predviđenih vjerojatnosti. Što je razlika manja, to smo sigurniji da smo poduzeli ispravnu akciju.
3. Izračunati diskontirane nagrade i pomnožiti gradijente s diskontiranim nagradama - to će osigurati da koraci s višim nagradama imaju veći utjecaj na konačni rezultat od onih s nižim nagradama.
4. Očekivane ciljne akcije za našu neuronsku mrežu djelomično će se uzimati iz predviđenih vjerojatnosti tijekom izvođenja, a djelomično iz izračunatih gradijenata. Koristit ćemo parametar `alpha` kako bismo odredili u kojoj mjeri se uzimaju u obzir gradijenti i nagrade - to se naziva *stopa učenja* algoritma za pojačanje.
5. Na kraju, treniramo našu mrežu na stanjima i očekivanim akcijama te ponavljamo proces.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Sada pokrenimo epizodu s renderiranjem kako bismo vidjeli rezultat:


In [ ]:
_ = run_episode(render=True)

Nadamo se da sada možete vidjeti da se štap prilično dobro može balansirati!

## Model Akter-Kritičar

Model Akter-Kritičar je daljnji razvoj politika gradijenata, u kojem gradimo neuronsku mrežu kako bismo naučili i politiku i procijenjene nagrade. Mreža će imati dva izlaza (ili to možete gledati kao dvije odvojene mreže):
* **Akter** će preporučiti akciju koju treba poduzeti dajući nam distribuciju vjerojatnosti stanja, kao u modelu politika gradijenata.
* **Kritičar** bi procijenio kakva bi nagrada bila od tih akcija. Vraća ukupne procijenjene nagrade u budućnosti za dano stanje.

Definirajmo takav model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Morali bismo malo izmijeniti naše funkcije `discounted_rewards` i `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Sada ćemo pokrenuti glavnu petlju za treniranje. Koristit ćemo ručni proces treniranja mreže izračunavanjem odgovarajućih funkcija gubitka i ažuriranjem parametara mreže:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Ključne točke

Vidjeli smo dva algoritma za pojačano učenje u ovom demo prikazu: jednostavni policy gradient i sofisticiraniji actor-critic. Možete primijetiti da ti algoritmi rade s apstraktnim pojmovima stanja, akcije i nagrade - zbog čega se mogu primijeniti na vrlo različite okruženja.

Pojačano učenje omogućuje nam da naučimo najbolju strategiju za rješavanje problema samo promatranjem konačne nagrade. Činjenica da nam nisu potrebni označeni skupovi podataka omogućuje nam da više puta ponavljamo simulacije kako bismo optimizirali naše modele. Ipak, još uvijek postoje mnogi izazovi u pojačanom učenju, koje možete istražiti ako odlučite posvetiti više pažnje ovom zanimljivom području umjetne inteligencije.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
